In [1]:
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Se retoma el momento maximo calculado en el punto 1.

Mmax = 1.16074707322184

In [3]:
perfiles = pd.read_excel('T2.xlsx', usecols="B:E", nrows=10, skiprows=3, index_col='perfil')
perfiles

,h [mm],b [mm],t [mm]
perfil,,,
1,25,50,5.0
2,30,60,5.5
3,35,70,6.0
4,40,80,7.0
5,50,100,8.5
6,55,115,9.0
7,60,120,10.0
8,70,140,11.5
9,80,160,13.0


In [4]:
# Datos del primer perfil.

h = perfiles.loc[1, 'h [mm]']
b = perfiles.loc[1, 'b [mm]']
t = perfiles.loc[1, 't [mm]']
h, b, t

(25, 50, 5.0)

In [5]:
# Area del primer perfil.

Area = (t * b) + (t * (h-t))
Area

350.0

In [6]:
# Coordenada Centroida Y

Centroide_Y = (((h - (t / 2)) * (b * t)) + ((h / 2) * (t * h)))/ Area
Centroide_Y

20.535714285714285

In [7]:
# Inercia en mm4
Iz = ((b * ((t) ** 3)) / 12) + ((t * ((h - t) ** 3)) / 12) + (((h - t) * t) * (((h - t) / 2) - Centroide_Y) ** 2) + ((b * t) * (((h - (t / 2)) - Centroide_Y) ** 2))
Iz

15918.898809523811

In [8]:
# distancia a la fibra superior.

y2 = h - Centroide_Y
y1 = Centroide_Y

In [9]:
# Calculo de esfuerzo en la fibra inferior [Mpa]

esfuerzo_inferior = (Mmax * 10 ** 6 * ( - Centroide_Y)) / Iz
esfuerzo_inferior

-1497.3881383932123

In [10]:
# Calculo de esfuerzo en la fibra superior [Mpa]

esfuerzo_superior = (Mmax * 10 ** 6 * (y2)) / Iz
esfuerzo_superior

325.51916052026365

In [11]:
# Areas
Resultado = perfiles
Resultado['Area [mm ^ 2]'] = (perfiles['b [mm]'] * perfiles['t [mm]']) + ((perfiles['h [mm]'] - perfiles['t [mm]']) * perfiles['t [mm]'])
Resultado

,h [mm],b [mm],t [mm],Area [mm ^ 2]
perfil,,,,
1,25,50,5.0,350.00
2,30,60,5.5,464.75
3,35,70,6.0,594.00
4,40,80,7.0,791.00
5,50,100,8.5,1202.75
6,55,115,9.0,1449.00
7,60,120,10.0,1700.00
8,70,140,11.5,2282.75
9,80,160,13.0,2951.00


In [12]:
# Centroides
Resultado['Yc [mm]'] = ((((perfiles['h [mm]'] - perfiles['t [mm]']) / 2) * ((perfiles['h [mm]'] - perfiles['t [mm]']) * perfiles['t [mm]'])) + (((perfiles['h [mm]'] - perfiles['t [mm]']) + (perfiles['t [mm]'] / 2)) * (perfiles['b [mm]'] * perfiles['t [mm]']))) / perfiles['Area [mm ^ 2]']


In [13]:
# Inercias

Resultado['Iz [mm ^ 4]'] = ((perfiles['b [mm]'] * ((perfiles['t [mm]']) ** 3)) / 12) + ((perfiles['t [mm]'] * ((perfiles['h [mm]'] - perfiles['t [mm]']) ** 3)) / 12) + (((perfiles['h [mm]'] - perfiles['t [mm]']) * perfiles['t [mm]']) * (((perfiles['h [mm]'] - perfiles['t [mm]']) / 2) - perfiles['Yc [mm]']) ** 2) + ((perfiles['b [mm]'] * perfiles['t [mm]']) * (((perfiles['h [mm]'] - perfiles['t [mm]']) + (perfiles['t [mm]'] / 2)) - perfiles['Yc [mm]']) ** 2)


In [14]:
Resultado['esfuerzo_superior [MPa]'] = Mmax * 10 ** 6 * (perfiles['h [mm]'] - perfiles['Yc [mm]']) / perfiles['Iz [mm ^ 4]']

In [15]:
Resultado['esfuerzo_inferior [MPa]'] = - Mmax * 10 ** 6 * perfiles['Yc [mm]'] / perfiles['Iz [mm ^ 4]']

# Respuesta del Inciso 1.

In [39]:
Tabla_1 = Resultado
Resultado

,h [mm],b [mm],t [mm],Area [mm ^ 2],Yc [mm],Iz [mm ^ 4],esfuerzo_superior [MPa],esfuerzo_inferior [MPa]
perfil,,,,,,,,
1,25,50,5.0,350.00,18.928571,1.501488e+04,469.360561,-1463.300572
2,30,60,5.5,464.75,22.900888,2.910029e+04,283.168117,-913.466475
3,35,70,6.0,594.00,26.873737,5.113253e+04,184.472302,-610.054144
4,40,80,7.0,791.00,30.659292,8.866585e+04,122.281577,-401.368567
5,50,100,8.5,1202.75,38.417845,2.115530e+05,63.548869,-210.790693
6,55,115,9.0,1449.00,42.642857,3.036222e+05,47.241336,-163.023570
7,60,120,10.0,1700.00,46.176471,4.318137e+05,37.158664,-124.125751
8,70,140,11.5,2282.75,53.935139,7.908459e+05,23.578856,-79.162143
9,80,160,13.0,2951.00,61.693833,1.337393e+06,15.888246,-53.545168


In [40]:
from openpyxl import load_workbook

with pd.ExcelWriter('T2.xlsx', mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:  
    book = load_workbook('T2.xlsx')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    Tabla_1.to_excel(writer, startrow=3, startcol=7, sheet_name='Perfiles', index=None)

# Solucion Inciso 2.

In [17]:
filtro = Resultado['esfuerzo_superior [MPa]'] <= 250
Resultado[filtro]

,h [mm],b [mm],t [mm],Area [mm ^ 2],Yc [mm],Iz [mm ^ 4],esfuerzo_superior [MPa],esfuerzo_inferior [MPa]
perfil,,,,,,,,
3,35,70,6.0,594.00,26.873737,5.113253e+04,184.472302,-610.054144
4,40,80,7.0,791.00,30.659292,8.866585e+04,122.281577,-401.368567
5,50,100,8.5,1202.75,38.417845,2.115530e+05,63.548869,-210.790693
6,55,115,9.0,1449.00,42.642857,3.036222e+05,47.241336,-163.023570
7,60,120,10.0,1700.00,46.176471,4.318137e+05,37.158664,-124.125751
8,70,140,11.5,2282.75,53.935139,7.908459e+05,23.578856,-79.162143
9,80,160,13.0,2951.00,61.693833,1.337393e+06,15.888246,-53.545168
10,90,180,15.0,3825.00,69.264706,2.186057e+06,11.009975,-36.778000


In [18]:
filtro = Resultado['esfuerzo_inferior [MPa]'] >= - 250
Resultado[filtro]

,h [mm],b [mm],t [mm],Area [mm ^ 2],Yc [mm],Iz [mm ^ 4],esfuerzo_superior [MPa],esfuerzo_inferior [MPa]
perfil,,,,,,,,
5,50,100,8.5,1202.75,38.417845,2.115530e+05,63.548869,-210.790693
6,55,115,9.0,1449.00,42.642857,3.036222e+05,47.241336,-163.023570
7,60,120,10.0,1700.00,46.176471,4.318137e+05,37.158664,-124.125751
8,70,140,11.5,2282.75,53.935139,7.908459e+05,23.578856,-79.162143
9,80,160,13.0,2951.00,61.693833,1.337393e+06,15.888246,-53.545168
10,90,180,15.0,3825.00,69.264706,2.186057e+06,11.009975,-36.778000


In [19]:
Resultado.loc[filtro, 'Area [mm ^ 2]']

perfil
5     1202.75
6     1449.00
7     1700.00
8     2282.75
9     2951.00
10    3825.00
Name: Area [mm ^ 2], dtype: float64

In [20]:
area_minima = Resultado.loc[filtro, 'Area [mm ^ 2]'].min()
area_minima

1202.75

In [21]:
indice = Resultado.loc[filtro, 'Area [mm ^ 2]'].idxmin()
indice

5

In [22]:
perfil_optimo = perfiles.loc[[indice], :'t [mm]']
perfil_optimo

,h [mm],b [mm],t [mm]
perfil,,,
5,50,100,8.5


# Eleccion del perfil Optimo.

In [28]:
Perfil_optimo = perfiles.loc[[indice], :'Area [mm ^ 2]']
Perfil_optimo

,h [mm],b [mm],t [mm],Area [mm ^ 2]
perfil,,,,
5,50,100,8.5,1202.75


In [31]:
from openpyxl import load_workbook

with pd.ExcelWriter('T2.xlsx', mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:  
    book = load_workbook('T2.xlsx')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    Perfil_optimo.to_excel(writer, startrow=18, startcol=6, sheet_name='Perfiles', index=None)

In [33]:
Perfil_optimo = perfiles.loc[[indice], 'esfuerzo_superior [MPa]' :'esfuerzo_inferior [MPa]']
Perfil_optimo

,esfuerzo_superior [MPa],esfuerzo_inferior [MPa]
perfil,,
5,63.548869,-210.790693


In [34]:
from openpyxl import load_workbook

with pd.ExcelWriter('T2.xlsx', mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:  
    book = load_workbook('T2.xlsx')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    Perfil_optimo.to_excel(writer, startrow=18, startcol=10, sheet_name='Perfiles', index=None)